In [9]:
import pandas as pd, numpy as np

pd.set_option("display.width", 5000)
pd.set_option("display.max_columns", 60)

%config InlineBackend.figure_format = 'retina'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.simplefilter('ignore')
pd.set_option('display.float_format', lambda x: '%.4f' % x)


In [10]:
data = pd.read_csv(
    "/Users/glebsokolov/Library/Group Containers/G69SCX94XU.duck/Library/Application Support/duck/Volumes/Dropbox GS/!Data Science/!Data Analyst Nanodegree/!Data-Analyst-Nanodegree/P2-Analyze-A-B-Test-Results/ab_data.csv"
)
countries = pd.read_csv(
    "/Users/glebsokolov/Library/Group Containers/G69SCX94XU.duck/Library/Application Support/duck/Volumes/Dropbox GS/!Data Science/!Data Analyst Nanodegree/!Data-Analyst-Nanodegree/P2-Analyze-A-B-Test-Results/countries.csv"
)
data.head()
countries.head()


,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


,user_id,country
0,834778,UK
1,928468,US
2,822059,UK
3,711597,UK
4,710616,UK


In [7]:
data.info()
countries.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290584 entries, 0 to 290583
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  290584 non-null  int64 
 1   country  290584 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.4+ MB


In [11]:
data = data.merge(countries, "left", on="user_id")
data.isna().sum()
print("Unique users count:")
data["user_id"].unique().shape[0]


user_id         0
timestamp       0
group           0
landing_page    0
converted       0
country         0
dtype: int64

Unique users count:


290584

In [33]:
import snoop
%load_ext snoop

In [12]:
corrupt = data.query(
    '(group=="control" and landing_page!="old_page") or (group=="treatment" and landing_page!="new_page")'
).index
data = data.drop(labels=corrupt, axis=0)


In [2]:
import scipy
from scipy import stats

In [56]:
temp = data.groupby("group")["user_id"].count()
control_size, treatment_size = temp["control"], temp["treatment"]
temp = (
    data.groupby(["group", "converted"])[["user_id"]]
    .count()
    .rename(columns={"user_id": "conversion_ratio"})
)
ind = temp.index
conversion_rate = temp.reset_index().apply(
    lambda x: x[["conversion_ratio"]] / control_size
    if x.group == "control"
    else x[["conversion_ratio"]] / treatment_size,
    axis=1,
)
conversion_rate.set_index(ind)


conversion_ratio
group     converted                  
control   0                    0.8796
          1                    0.1204
treatment 0                    0.8812
          1                    0.1188

In [33]:
def calculate_proportion_difference(data):
    sample1, sample2 = data.query("group=='control'")["converted"], data.query("group=='treatment'")["converted"]
    p1, p2 = float(sum(sample1)) / len(sample1), float(sum(sample2)) / len(sample2)
    return p1 - p2

In [38]:
def proportions_confint_diff(sample1, sample2, alpha=0.05):
    z = stats.norm.ppf(1 - alpha / 2.0)
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)

    left_boundary = (p1 - p2) - z * np.sqrt(
        p1 * (1 - p1) / len(sample1) + p2 * (1 - p2) / len(sample2)
    )
    right_boundary = (p1 - p2) + z * np.sqrt(
        p1 * (1 - p1) / len(sample1) + p2 * (1 - p2) / len(sample2)
    )
    return (round(left_boundary * 100, 3), round(right_boundary * 100, 3))

In [39]:
control, treatment = data.query('group=="control"'), data.query('group=="treatment"')
proportions_confint_diff(control['converted'], treatment['converted'])

(-0.078, 0.394)